# PHYS-2260 Final Project: Signatures with Complex Fourier Transforms

### Authors: Andrew Antenberg and Ethan Lior

### Imports

In [ ]:
!pip install pygame
!pip install pygame_widgets
!pip install svgpathtools

: 

In [ ]:
import pygame
import pygame_widgets
from pygame_widgets.slider import Slider
from pygame_widgets.textbox import TextBox
from svgpathtools import svg2paths
from numpy import cos, sin, pi, sqrt, arctan2
import pygame.draw as draw
from pygame import gfxdraw
import numpy as np
from random import random
import pygame.locals
from functools import reduce

: 

### Applying the Fourier Transform

In [ ]:
def fourier_term(signals, k):
  N = len(signals)
  X_k = complex()
  for n in range(N):
    theta = 2 * pi * k * n / N
    X_k += complex(cos(theta), -sin(theta)) * signals[n]
  return X_k / N

def fourier_transform(signals):  
  return [fourier_term(signals, k) for k in range(len(signals))]

: 

In [ ]:
class Wave:
  def __init__(self, frequency, amplitude, phase):
    self.frequency = frequency
    self.amplitude = amplitude
    self.phase = phase
  def __repr__(self):
    return f"Wave(freq={self.frequency}, amp={self.amplitude}, phase={self.phase})"

  @staticmethod
  def from_fourier_term(k, cmplx_num):
    r = cmplx_num.real
    i = cmplx_num.imag
    return Wave(k, sqrt(r * r + i * i), arctan2(i, r))

def generate_wave_info(signals):
  fourier_terms = fourier_transform(signals)
  return [Wave.from_fourier_term(k, term) for k, term in enumerate(fourier_terms)]

def generate_waves(points_x, points_y):
  print('Generating fourier waves from vertices.')
  x_waves = sorted(generate_wave_info(points_x), key=lambda x: -x.amplitude)
  y_waves = sorted(generate_wave_info(points_y), key=lambda x: -x.amplitude)
  print('Waves generated!')
  return x_waves, y_waves

: 

### Importing data for each letter and formatting into dictionary

In [ ]:
def read_file(filename):
  file = open(filename, "r")
  return [[float(num) for num in line.split(", ")] for line in file]

: 

Function to extract data from svg file and format into x and y position arrays

In [ ]:
def extract(paths):
    xs = []
    ys = []
    for path in paths:
        for segment in path:
            # Extract coordinates from the segment
            for point in segment:
                x, y = point.real, point.imag
                xs.append(x)
                ys.append(y)
    return xs,ys

def get_svg_coords(filename):
  paths, _ = svg2paths(filename)
  return extract(paths)

: 

### Generating the map from letter to list of vertices

In [ ]:
alphabet = [chr(65 + i) for i in range(26)] + [chr(97 + i) for i in range(26)]
dc = dict()
for letter in alphabet:
    is_uppercase = letter < 'a'
    filepath = f"assets/letters/{'upper' if is_uppercase else 'lower'}/{letter}.svg"
    xs, ys = get_svg_coords(filepath)
    dc[letter] = (xs, ys)

letter_height = reduce(lambda acc, curr: max(max(dc[curr][1]), acc), dc, 0)
dc[' '] = [0, 30], [letter_height, letter_height]

: 

### Generating the vertices for an input string

In [ ]:
# Initializing array for x and y coordinates
def generate_text_vertices(word):   
    print(f'Generating vertices of "{word}"')
    points_x = []
    points_y = []
    offset = 0
    for i in word:
        points_x.append(offset)
        points_y.append(letter_height)
        points_x += [x + offset for x in dc[i][0]]
        curr_max_y = max(dc[i][1])
        points_y += [letter_height - curr_max_y + y for y in dc[i][1]]
        points_x.append(points_x[-1])
        points_y.append(letter_height)
        offset = max(points_x)
    points_x = [point - offset / 2 for point in points_x]
    points_y = [point - letter_height / 2 for point in points_y]
    print('Vertices generated!')
    return points_x, points_y


: 

In [ ]:
class Point:
  def __init__(self, x, y):
    self.x = x
    self.y = y
  
  def __add__(self, other):
    return Point(self.x + other.x, self.y + other.y)
  
  def __sub__(self, other):
    return Point(self.x - other.x, self.y - other.y)
  
  def __iadd__(self, other):
    self.x += other.x
    self.y += other.y
    return self

  def __truediv__(self, num):
    return Point(self.x / num, self.y / num)
  
  def __mul__(self, num):
    return Point(self.x * num, self.y * num)
  
  def __repr__(self):
    return f"({self.x}, {self.y})"
  
  def __eq__(self, other):
    return self.x == other.x and self.y == other.y
  
  def __ne__(self, other):
    return not self.__eq__(other)
  
  def __hash__(self):
    return hash((self.x, self.y))
  
  def as_tuple(self):
    return (self.x, self.y)
  
  def as_ints(self):
    return (int(self.x), int(self.y))
  
  def on_grid(self, grid):
    col, row = self.as_ints()
    return grid[row][col]
  
  def set_grid_value(self, grid, value):
    col, row = self.as_ints()
    grid[row][col] = value

: 

### Visualizing a fourier transform

In [ ]:
def generate_and_display_fourier_transform(x_coords, y_coords, grid=None):
    xwaves, ywaves = generate_waves(x_coords, y_coords)
    display_fourier_transform(xwaves, ywaves, grid)

def display_fourier_transform(x_waves, y_waves, grid=None):
    pygame.init()

    background_color = (255, 255, 255)
    circle_color = (100, 100, 100)
    radius_color = (200, 200, 200)
    pointer_line_color = (255, 100, 100)

    old_line_color = (100, 100, 100)
    drawing_color = (0, 0, 50)
    widget_color = (0, 122, 255)

    screensize = Point(1280, 720)
    screencenter = screensize / 2
    screen = pygame.display.set_mode(screensize.as_tuple())
    clock = pygame.time.Clock()
    running = True
    time = 0
    points = []
    oldPoints = []
    epsilon = pi / 8

    slider_x = screencenter.x - 50
    slider = Slider(screen, slider_x, 600, 100, 10, min=1, max=5, step=1, initial=1, handleColour=widget_color)

    font = pygame.font.Font(None, 24)
    info_text = get_info_text(x_waves, y_waves)
    line_spacing = 40
    show_metadata = False

    output = TextBox(screen, slider_x, 630, 0, 0, fontSize=12)
    speed_label = TextBox(screen, slider_x, 642, 0, 0, fontSize=12)
    speed_label.setText('(Higher Speed = Lower Resolution)')
    speed_label.disable()
    output.disable()

    while running:
        output.setText(f'Speed: {slider.getValue()}x')

        events = pygame.event.get()
        # poll for events
        # pygame.QUIT event means the user clicked X to close your window
        for event in events:
            if event.type == pygame.QUIT:
                running = False
                pygame.quit()
            elif event.type == pygame.locals.KEYDOWN:
                if event.key == pygame.locals.K_SPACE:
                    show_metadata = not show_metadata

        # fill the screen with a color to wipe away anything from last frame
        screen.fill(background_color)

        
        if grid is not None:
            box_size = 10
            grid_top_left = screencenter - (Point(len(grid[0]) * box_size, len(grid * box_size)) / 2)
            draw.rect(screen, (100, 100, 100), (grid_top_left.as_tuple(), (Point(len(grid[0]), len(grid)) * box_size).as_tuple()), width=2)
            for row_index, row in enumerate(grid):
                for col_index, v in enumerate(row):
                    if v == 1:
                        draw.rect(screen, (0, 0, 0), ((grid_top_left + Point(col_index * box_size, row_index * box_size)).as_tuple(), (box_size, box_size)))

        if show_metadata:
            for i, line in enumerate(info_text):
                text_surface = font.render(line, True, (0, 0, 0))
                screen.blit(text_surface, (screensize.x - 250, 20 + i * line_spacing))

        currPointX = Point(screencenter.x, 100)
        for wave in x_waves:
            prevPoint = currPointX

            freq = wave.frequency
            radius = wave.amplitude
            phase = wave.phase

            offsetPoint = Point(cos(freq * time + phase), sin(freq * time + phase)) * radius
            currPointX = prevPoint + offsetPoint
            draw.aaline(screen, radius_color, prevPoint.as_tuple(), currPointX.as_tuple(), 2)
            radiusPoint = Point(radius, radius)
            topLeft = prevPoint - radiusPoint
            gfxdraw.aacircle(screen, int(prevPoint.x), int(prevPoint.y), int(radius), circle_color)
        
        currPointY = Point(100, screencenter.y)
        for wave in y_waves:
            prevPoint = currPointY

            freq = wave.frequency
            radius = wave.amplitude
            phase = wave.phase

            offsetPoint = Point(cos(freq * time + phase + (pi / 2)), sin(freq * time + phase + (pi / 2))) * radius
            currPointY = prevPoint + offsetPoint
            draw.aaline(screen, radius_color, prevPoint.as_tuple(), currPointY.as_tuple(), 2)
            radiusPoint = Point(radius, radius)
            topLeft = prevPoint - radiusPoint
            gfxdraw.aacircle(screen, int(prevPoint.x), int(prevPoint.y), int(radius), circle_color)

        time += slider.getValue() * (2 * pi / len(x_waves))
            
        finalPoint = (currPointX.x, currPointY.y)
        draw.aaline(screen, pointer_line_color, currPointX.as_tuple(), finalPoint)
        draw.aaline(screen, pointer_line_color, currPointY.as_tuple(), finalPoint)

        
        points.append((currPointX.x, currPointY.y))
        if (time >= 2 * pi - epsilon):
            oldPoints.append(points[0])
            points = points[1:]
        if len(oldPoints) >= 250:
            oldPoints = oldPoints[1:]

        if len(oldPoints) > 1:
            draw.aalines(screen, old_line_color, False, oldPoints)
        if len(points) > 1:
            draw.aalines(screen, drawing_color, False, points)
        
        draw.circle(screen, widget_color, (currPointX.x, currPointY.y), 5)

        
        pygame_widgets.update(events)
        pygame.display.flip()

        clock.tick(60)  # limits FPS to 60

    pygame.quit()

def get_info_text(xwaves, ywaves):
    num_waves = len(xwaves)
    return f"""General info:
        {num_waves} Vertices
        {num_waves} Waves
X-Values:
        Max Radius: {max(xwaves, key=lambda x: x.amplitude).amplitude:.2f}
        Min Radius: {min(xwaves, key=lambda x: x.amplitude).amplitude:.2e}
        Max Frequency: {max(xwaves, key=lambda x: x.frequency).frequency}
        Min Frequency: {min(xwaves, key=lambda x: x.frequency).frequency}
Y-Values:
        Max Radius: {max(ywaves, key=lambda x: x.amplitude).amplitude:.2f}
        Min Radius: {min(ywaves, key=lambda x: x.amplitude).amplitude:.2e}
        Max Frequency: {max(ywaves, key=lambda x: x.frequency).frequency}
        Min Frequency: {min(ywaves, key=lambda x: x.frequency).frequency}""".split('\n')

: 

## Brownian Motion

In [ ]:
GRID_ROWS = 10
GRID_COLS = 10

INIT_ROW = 0
INIT_COL = 0


def generate_maze(grid_rows=GRID_ROWS, grid_cols=GRID_COLS, init_row=INIT_ROW, init_col=INIT_COL):
  print('Generating maze.')
  grid = [[0 if i % 2 == 0 and j % 2 == 0 else 1 for i in range(grid_rows * 2 - 1)] for j in range(grid_cols * 2 - 1)]
  visited = [[False for _ in row] for row in grid]
  stack = [Point(init_col, init_row) * 2]

  while len(stack) != 0:
    p = stack.pop()
    row = int(p.y)
    col = int(p.x)
    visited[row][col] = True
    unvisited_neighbors = get_unvisited_neighbors(row, col, visited)
    if len(unvisited_neighbors) != 0:
      stack.append(p)
      random_neighbor_index = int(random() * len(unvisited_neighbors))
      random_neighbor = unvisited_neighbors[random_neighbor_index]
      random_neighbor.set_grid_value(visited, 0)
      in_between = (p + random_neighbor) / 2
      in_between.set_grid_value(grid, 0)
      stack.append(random_neighbor)
  print('Maze generated!')
  return grid

def get_unvisited_neighbors(row, col, visited):
  return get_unvisited_neighbor_points(Point(col, row), visited)

def get_unvisited_neighbor_points(point, visited):
  num_rows = len(visited)
  num_cols = len(visited[0])
  offsets = [Point(-2, 0), Point(2, 0), Point(0, -2), Point(0, 2)]
  result = []
  for offset in offsets:
      new_p = point + offset
      if (0 <= new_p.y < num_rows) and (0 <= new_p.x < num_cols) and not new_p.on_grid(visited):
        result.append(new_p)
  return result

: 

In [ ]:
def random_offset():
  rand = random()
  if rand < 0.25:
    return Point(2, 0)
  elif rand < 0.5:
    return Point(-2, 0)
  elif rand < 0.75:
    return Point(0, 2)
  else:
    return Point(0, -2)
  
def solve_maze_brownian(maze, num_movers=10):
  movers = [Point(INIT_COL, INIT_ROW) for _ in range(num_movers)]
  paths = [[] for _ in range(num_movers)]
  end_pos = Point(len(maze) - 1, len(maze[0]) - 1)
  for mover_index, mover in enumerate(movers):
    while mover != end_pos:
      if mover == Point(0, 0):
        paths[mover_index] = []
      offset = random_offset()
      new_pos = mover + offset
      avg_pos = (new_pos + mover) / 2
      if 0 <= new_pos.x < len(maze[0]) and 0 <= new_pos.y < len(maze) and avg_pos.on_grid(maze) == 0:
        mover = new_pos
        paths[mover_index].append(avg_pos)
        paths[mover_index].append(new_pos)

  min_path = paths[0]
  for path in paths:
    if len(path) < len(min_path):
      min_path = path
  return min_path

def solve_maze_bfs(maze):
  print('Solving maze')
  visited = [[False for _ in row] for row in maze]
  init = Point(0, 0)
  parent = { init: None }
  queue = [Point(0, 0)]
  while len(queue) != 0:
    curr = queue.pop(0)
    curr.set_grid_value(visited, True)
    for new_point in get_unvisited_neighbor_points(curr, visited):
      avg_pos = (curr + new_point) / 2
      if 0 <= new_point.x < len(maze[0]) and 0 <= new_point.y < len(maze) and not new_point.on_grid(visited) and avg_pos.on_grid(maze) == 0:
        queue.append(new_point)
        parent[new_point] = curr
  print('Finished exploring. Building path.')
  path = []
  pt = Point(len(maze[0]) - 1, len(maze) - 1)
  while pt is not None:
    path.append(pt)
    pt = parent[pt]
  print('Path built!')
  return path[::-1]



def extract_coords(path):
  return [pos.x for pos in path], [pos.y for pos in path]

: 

In [ ]:
def generate_maze_path_coords(maze, solve_maze_fn=solve_maze_brownian):
  path = solve_maze_fn(maze)
  xs, ys = extract_coords(path)
  xs = [(x - (len(maze[0]) / 2)) * 10 + 5 for x in xs]
  ys = [(y - (len(maze) / 2)) * 10 + 5 for y in ys]

  confetti_xs, confetti_ys = get_svg_coords('assets/confetti.svg')
  confetti_xs.append(confetti_xs[0])
  confetti_ys.append(confetti_ys[0])

  xs += [x + len(maze[0]) * 10 + 10 for x in confetti_xs] + [(x - (len(maze[0]) / 2)) * 10 + 5 for x in [len(maze[0]), -1, -1]]
  ys += confetti_ys + [(y - (len(maze) / 2)) * 10 + 5 for y in [len(maze), len(maze), -1]]
  return xs, ys

: 

In [ ]:
def start_text_interaction():
  text = input('What would you like to be written?\n') #Taking in user input
  points_x, points_y = generate_text_vertices(text)
  generate_and_display_fourier_transform(points_x, points_y)

def start_maze_solver_interaction():
  default_dim = 10
  maze_dim = input(f'What size maze would you like to generate? (Recommended: {default_dim})')
  parsed_maze_dim = 10
  try:
    parsed_maze_dim = int(maze_dim)
  except ValueError:
    print(f'"{maze_dim}" is not a number. Using {default_dim} instead')
  maze = generate_maze(parsed_maze_dim, parsed_maze_dim)
  solver_type = input(f'What algorithm would you like to use to solve the maze? 1. Brownian Motion\n2. BFS  (Default: brownian)')
  solver_fn = solve_maze_bfs if len(solver_type) > 0 and solver_type[0] == '2' else solve_maze_brownian
  points_x, points_y = generate_maze_path_coords(maze, solve_maze_fn=solver_fn)
  generate_and_display_fourier_transform(points_x, points_y, maze)

def start_svg_interaction():
  svg_path = input('Please type the filepath of your svg')
  try:
    svg_xs, svg_ys = get_svg_coords(svg_path)
    generate_and_display_fourier_transform(svg_xs, svg_ys)
  except FileNotFoundError:
    print(f'Invalid file path: "{svg_path}". Please try again!')

: 

## Play Around!

In [ ]:
interaction_type = input('What would you like to visualize?\n1. Text\n2. Maze Solver\n3. SVG File')
match interaction_type[0]:
  case '1':
    start_text_interaction()
  case '2':
    start_maze_solver_interaction()
  case '3':
    start_svg_interaction()

: 